# Using the OSF API

For more information, visit the full [OSF API docs](http://developer.osf.io)!

We'll be using the staging version of the OSF and API for this tutorial. Because staging is always in active development, if the endpoints fail to work at any point, feel free to switch to production OSF! Just note that you'll have to create a new token, and that any test work you make public will be available to anyone! Simply remove "staging" from the base STAGING_OSF_API url listed below for production OSF endpoints.

Before starting this tutorial, make sure to [create an account on the staging version of the osf](https://staging.osf.io), login to that account, and create an API token by [visitng your settings page](https://staging.osf.io/settings/tokens/).

Save your token as an enviornment variable, or replace the enviornment variable below with the text version of your token for local testing.

## Create a Project, Upload a File

Here's an example of how to create a project (called a node) on the OSF, and then follow the API relationships to upload a file.

This is a python implementation of a guide found on the OSF [detailing a typical OSF Workflow](https://osf.io/y9jdt/wiki/Typical%20Workflow/)

In [ ]:
import os
import json
import requests

STAGING_OSF_TOKEN = os.environ['STAGING_OSF_TOKEN']  # replace this line with your token instead if you like
STAGING_OSF_API = 'https://staging-api.osf.io/v2/'

In [ ]:
# Let's defne a few helper functions to make sending credentials easier
def post_request(url, data, auth):
    headers = {'Content-Type': 'application/vnd.api+json'}
    if auth:
        headers['Authorization'] = 'Bearer {}'.format(auth)
    data = json.dumps(data)
    return requests.post(url, headers=headers, data=data)

In [ ]:
def get_request(url, auth=None):
    headers = {'Authorization': 'Bearer {}'.format(auth)}
    return requests.get(url, headers=headers)

In [ ]:
def put_request(url, data, auth):
    headers = {
        'Content-Type': 'application/vnd.api+json',
        'Authorization': 'Bearer {}'.format(auth)
    }
    data = json.dumps(data)
    return requests.put(url, headers=headers, data=data)

In [ ]:
# Define the data for the node we'd like to create
node_data = {
    "data": {
        "type":"nodes",
        "attributes": {
            "title":"Testing Example",
            "description": "This is a node created as an example of how to create a node!",
            "public": False,
            "category":"project"
        }
    }
}

In [ ]:
# Post the data, get a response back with details about our node
node_response = post_request(STAGING_OSF_API + 'nodes/', node_data, STAGING_OSF_TOKEN)
print(json.dumps(node_response.json(), indent=4))

In [ ]:
# Find the files relationship, follow the related -> href link
files_link = node_response.json()['data']['relationships']['files']['links']['related']['href']
files_response = get_request(files_link, STAGING_OSF_TOKEN).json()
print(json.dumps(files_response, indent=4))

In [ ]:
# Find the upload link for OSF Storage in that list - should be the first element in the list for new nodes
# A node can have several external storage providers configured
upload_link = files_response['data'][0]['links']['upload']
upload_link

In [ ]:
# Upload the file along with the kind and file name

upload_link_with_filename = upload_link + '?kind=file&name=newest_file.txt'
file_data = 'This is the entirety of the contents of the file I am uploading. It could have been more, but for an example, a small file seems like a better idea.'

put_response = put_request(upload_link_with_filename, file_data, STAGING_OSF_TOKEN)
print(json.dumps(put_response.json(), indent=4))

## You did it!
Visit your project on the OSF and see your newly updated file!

In [ ]:
# Check our your project on the OSF by visiting the project's link
node_response.json()['data']['links']['html']

# Querying the SHARE API

In [ ]:
SHARE_API_BASE = 'https://share.osf.io/api/v2/'

In [ ]:
# Get the total number of SHARE sources
sources_query = requests.get(SHARE_API_BASE + 'sources').json()
count = sources_query['meta']['pagination']['count']
print('There are {} sources in SHARE'.format(count))

In [ ]:
# Get the total number of creativeworks in SHARE
creativeworks_search = 'search/creativeworks/_search'
base_search = requests.get(SHARE_API_BASE + creativeworks_search).json()
total_creativeworks = base_search['hits']['total']
print('There are {} works in SHARE'.format(total_creativeworks))

In [ ]:
# Print out the first 10 titles
results = base_search['hits']['hits']

for result in results:
    print(result['_source']['title'])

In [ ]:
def post_query(url, query):
    headers = {'Content-Type': 'application/json'}
    data = json.dumps(query)
    return requests.post(url, headers=headers, data=data)

In [ ]:
# Get query forming hints by searching https://share.osf.io/discover
search_query = {
    "query": {
        "bool": {
            "must": {
                "query_string": {
                    "query": "climate change"
                }
            },
            "filter": [
                {
                    "term": {
                        "types": "software"
                    }
                }
            ]
        }
    }
}

In [ ]:
software_results = post_query(SHARE_API_BASE + creativeworks_search, search_query).json()

# Let's check out the details of the first result
print(json.dumps(software_results['hits']['hits'][0]['_source'], indent=4))

In [ ]:
# Iterate through the first page of results, print each title
for result in software_results['hits']['hits']:
    print(result['_source']['title'])